# Introduction to Message Passing Interface


# What is MPI?

* Message Passing Interface
  * Most useful on distributed memory machines
  * The *de facto standard* parallel programming interface
  
* Many implementations, interfaces in C/C++, Fortran, Python via MPI4Py


# Message Passing Paradigm

* A parallel MPI program is launched as separate processes (tasks), each with thier own address space.
  * Requires partitioning data across tasks.
* **Data is explicitly** moved from task to task 
  * A task accesses the data of another task through a transaction called "message passing" in which a copy of the
data (message) is transferred (passed) from one task to another.
* There are two classes of message passing
  * **Point-to-point** involve only two tasks
  * **Collective messages** involve a set of tasks


# MPI4Py

 * MPI4Py provides an interface very similar to the MPI standard C++ interface
 * You can communicate Python objects.
 * What you may lose in performance, you gain in shorter development time


# Communicators

 * MPI uses communicator objects to identify a set of processes which communicate only within their set.
 * `MPI_COMM_WORLD` is defined as all processes (ranks) of your job.
 * Usually required for most MPI calls 
 * Rank
   * Unique process ID within a communicator
   * Assigned by the system when the process initalizes
   * Used to specify sources and destinations of messages


In [ ]:
%%file hello.py
#!/usr/bin/env python
from mpi4py import MPI
comm = MPI.COMM_WORLD
print("Hello, World! My rank is: " + str(comm.rank))

In [ ]:
!mpiexec -np 2 python hello.py

# Point-to-Point Communication

Sending data from one point (process/task) to another point (process/task)


In [ ]:
%%file send_recv.py
#!/usr/bin/env python
from mpi4py import MPI
import numpy
comm = MPI.COMM_WORLD
rank = comm.rank
size = comm.size
v = numpy.array([rank] * 10,dtype=float)
if comm.rank == 0:
    comm.send(v, dest = (rank + 1) % size)
if comm.rank > 0:
    data = comm.recv(source = (rank - 1) % size)
    comm.send(v, dest = (rank + 1) % size)
if comm.rank == 0:
    data = comm.recv(source = size - 1)
print("My rank is " + str(rank) + "\n I received this:\n" + str(data))

In [ ]:
!mpiexec -np 2 python send_recv.py

# Collective Communication


<table>
    <tr style="background-color:transparent">
        <td><img src="images/collective_comm.png" width=400/></td>
    </tr>
</table>
[Image Source](https://computing.llnl.gov/tutorials/parallel_comp/images/collective_comm.gif)


# Scatter

Rank 0 acts as a leader, creating a list and `scatter`ing it out to all
ranks evenly


In [ ]:
%%file scatter.py
#!/usr/bin/env python
from mpi4py import MPI
import numpy
sendbuf = []
comm = MPI.COMM_WORLD
if comm.rank == 0:
    m = numpy.random.randn(comm.size, comm.size)
    print("Original array on rank 0:\n" + str(m))
    sendbuf = m
v = comm.scatter(sendbuf, root=0)
print("I got this array:\n" + str(v))

In [ ]:
!mpiexec -np 2 python scatter.py

# Gather

`gather` is a command that collects results from all processes into a list.


In [ ]:
%%file gather.py
#!/usr/bin/env python
from mpi4py import MPI
import numpy
comm = MPI.COMM_WORLD
sendbuf = []
root = 0
if comm.rank == 0:
    m = numpy.array(range(comm.size * comm.size), dtype=float)
    m.shape=(comm.size, comm.size)
    print("Original array on rank 0:\n" + str(m))
    sendbuf = m
    
v = comm.scatter(sendbuf, root=0)
print("I got this array:\n" + str(v))
v = v * v
recvbuf = comm.gather(v, root=0)
if comm.rank == 0:
    print("New array on rank 0:\n" + str(numpy.array(recvbuf)))

In [ ]:
!mpiexec -np 2 python gather.py

# Broadcast

`bcast` sends a single object to every process.


In [ ]:
%%file broadcast.py
#!/usr/bin/env python
from mpi4py import MPI
import numpy
comm = MPI.COMM_WORLD
sendbuf = []
if comm.rank == 0:
    m = numpy.array(range(comm.size * comm.size), dtype=float)
    m.shape=(comm.size, comm.size)
    print("Original array on rank 0:\n" + str(m))
    sendbuf = m
    
v = comm.scatter(sendbuf, root=0)
print("I got this array:\n" + str(v))
v = v * v
recvbuf = comm.gather(v, root=0)
if comm.rank == 0:
    print("New array on rank 0:\n" + str(numpy.array(recvbuf)))
if MPI.COMM_WORLD.rank == 0:
    sendbuf = "Done!"
recvbuf = MPI.COMM_WORLD.bcast(sendbuf, root=0)
print(recvbuf)

In [ ]:
!mpiexec -np 2 python broadcast.py

# Reduce

`reduce` performs a parallel reduction operation
 * The default is summation, but many other operators are available.


In [ ]:
%%file reduce.py
#!/usr/bin/env python
from mpi4py import MPI
import numpy
comm = MPI.COMM_WORLD
sendbuf = []
if comm.rank == 0:
    m = numpy.array(range(comm.size * comm.size), dtype=numpy.float)
    m.shape=(comm.size, comm.size)
    print("Original array on rank 0:\n" + str(m))
    sendbuf = m
v = comm.scatter(sendbuf, root=0)
print("I got this array:\n" + str(v))
recvbuf = comm.reduce(v, root=0)
if comm.rank == 0:
    total_sum = numpy.sum(numpy.array(recvbuf))
    print("New array on rank 0:\n" + str(total_sum))

In [ ]:
!mpiexec -np 2 python reduce.py

# References

* http://mpi4py.scipy.org/docs/usrman/index.html

* https://computing.llnl.gov/tutorials/parallel_comp/


In [3]:
%%javascript
function hideElements(elements, start) {
    for(var i = 0, length = elements.length; i < length;i++) {
        if(i >= start) {
            elements[i].style.display = "none";
        }
    }
}

var prompt_elements = document.getElementsByClassName("prompt");
hideElements(prompt_elements, 0)

<IPython.core.display.Javascript object>